# Scrape argentinian politics data to fine tune bert
[Reddit's Api Methods](https://praw.readthedocs.io/en/latest/code_overview/models/submission.html)

[My Reddit's Api](https://www.reddit.com/prefs/apps)


### Import origial documents

In [15]:
path_to_MyModule = '..'

import sys
sys.path.insert(0, path_to_MyModule) 

from MyModule.GeneralFunctions import *

In [16]:
df = pd.read_excel('..\datos.xlsx')[['ID','texto','desafio']]

In [17]:
# Limpieza
df.drop_duplicates(subset='texto', inplace=True)

# Quitando texto de mas en columna "desafio"
df['desafio'] = df['desafio'].apply(lambda x: re.findall('[0-9]+', x)[0])

# A str
df['texto'] = df['texto'].astype(str)

In [18]:
pp_object = Preprocess(lemma=False, stopwords=False)
documents = df['texto'].values.tolist()
documents = pp_object.preprocess(documents)
# documents = [doc for doc in documents if doc != '']

### Scrape data

In [99]:
import pandas as pd
import praw

In [100]:
# Instanciate api
user_agent = 'Scraper 1.0 by /u/felipeaguirre'

reddit = praw.Reddit(
    client_id = 'H3Bsol4shCPuWVjcDtWXDw',
    client_secret = 'dgPGF2jYXuVWfR-l7WDV87wFxOFj1g',
    user_agent = user_agent,
)

In [101]:
# Scrape hot subbredit
subreddits = ['RepublicaArgentina', 'argentina','Republica_Argentina']

titulos = []
contenido = []
comentarios = []

for sb in subreddits:
    print(f'Doing {sb}')
    results = reddit.subreddit(sb).hot(limit=None)
    
    for r in results:
        try:
            titulos.append(r.title)
            contenido.append(r.selftext)
            for comment in r.comments:
                comentarios.append(comment.body)
        except:
            pass

Doing RepublicaArgentina
Doing argentina
Doing Republica_Argentina


In [102]:
# # Save
# with open('new_result_raw.txt', 'w') as f:
#     for i in titulos:
#         f.write(i + '\n')
#     for i in contenido:
#         f.write(i + '\n')
#     for i in comentarios:
#         f.write(i + '\n')

### Keep only spanish texts

In [7]:
with open('./training_data/reddit_raw.txt', 'r') as f:
    data = f.readlines()

In [9]:
# Keep only spanish texts
from langdetect import detect

data_sp = []
for d in data:
    try:
        if detect(d) == 'es':
            data_sp.append(d)
    except:
        pass

In [10]:
len(data_sp)

44541

### Keep only texts that contain important words in original documents

In [9]:
pp_data = pp_object.preprocess(data_sp)

In [124]:
unique_words = list(set(' '.join(documents).split(' ')))

['responsabilidad',
 'identificar',
 'cargo',
 'funcion',
 'casi',
 'brindar',
 'centro',
 'epoca',
 'honestamente',
 'esplendido',
 'alcance',
 'empresa',
 'feria',
 'inteligente',
 'critica',
 'internacional',
 'disminuir',
 'mentira',
 'caer',
 'emocion',
 'anterior',
 'perfecto',
 'accion',
 'ano',
 'contradecir',
 'minoria',
 'izquierdo',
 'veracidad',
 'interes',
 'posicion',
 'problematica',
 'normal',
 'libremercado',
 'gustariar',
 'faltarian',
 'tramo',
 'conectar',
 'desigualdad',
 'seguro',
 'viejo',
 'rearmar',
 'poder',
 'acusar',
 'resaltar',
 'concepto',
 'ir',
 'socioeconomico',
 'desacuerdo',
 'empleado',
 'plata',
 'impartir',
 'auto',
 'otorgar',
 'desincentivo',
 'exencion',
 'fundar',
 'huelga',
 'debatir',
 'preferir',
 'rentable',
 'fuerte',
 'inmersivo',
 'teqball',
 'controversia',
 'resolucion',
 'capitslista',
 'educacional',
 '18',
 'voler',
 'huerta',
 'cansado',
 'vinculado',
 'olmedo',
 'verdad',
 'relacionar',
 'aire',
 'desempleado',
 'garantizandolo',

In [126]:
final_data = []

for i, d in enumerate(pp_data):
    for w in unique_words:
        if w in d:
            final_data.append(data_sp[i])
            break

In [132]:
# Save
# with open('result_pp.txt', 'w') as f:
#     for i in final_data:
#         f.write(i)

### BERT similarity

In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

In [19]:
pp_data = pp_object.preprocess(data_sp)

In [20]:
original_data_vectors = model.encode(documents)
trainig_data_vectors = model.encode(pp_data)

In [21]:
simil_docs = []
for i, x in enumerate(trainig_data_vectors):
    x = x.reshape(1, -1)
    if i%500==0:print(f'Vamos por el {i} de {len(trainig_data_vectors)}.')
    for i2, x2 in enumerate(original_data_vectors):
        x2 = x2.reshape(1, -1)
        similarity = cosine_similarity(x, x2)
        if similarity >= 0.6:
            simil_docs.append(data_sp[i])
            break

Vamos por el 0 de 44541.
Vamos por el 500 de 44541.
Vamos por el 1000 de 44541.
Vamos por el 1500 de 44541.
Vamos por el 2000 de 44541.
Vamos por el 2500 de 44541.
Vamos por el 3000 de 44541.
Vamos por el 3500 de 44541.
Vamos por el 4000 de 44541.
Vamos por el 4500 de 44541.
Vamos por el 5000 de 44541.
Vamos por el 5500 de 44541.
Vamos por el 6000 de 44541.
Vamos por el 6500 de 44541.
Vamos por el 7000 de 44541.
Vamos por el 7500 de 44541.
Vamos por el 8000 de 44541.
Vamos por el 8500 de 44541.
Vamos por el 9000 de 44541.
Vamos por el 9500 de 44541.
Vamos por el 10000 de 44541.
Vamos por el 10500 de 44541.
Vamos por el 11000 de 44541.
Vamos por el 11500 de 44541.
Vamos por el 12000 de 44541.
Vamos por el 12500 de 44541.
Vamos por el 13000 de 44541.
Vamos por el 13500 de 44541.
Vamos por el 14000 de 44541.
Vamos por el 14500 de 44541.
Vamos por el 15000 de 44541.
Vamos por el 15500 de 44541.
Vamos por el 16000 de 44541.
Vamos por el 16500 de 44541.
Vamos por el 17000 de 44541.
Vamos por